# 경사 하강법을 이용한 얕은 신경망 학습

In [3]:
import tensorflow as tf
import numpy as np

## 하이퍼 파라미터 설정

In [4]:
EPOCHS = 1000

## 네트워크 구조 정의

### 얕은 신경망

In [19]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel,self).__init__()
        self.d1 = tf.keras.layers.Dense(128,input_dim=2,activation='sigmoid')
        self.d2 = tf.keras.layers.Dense(10,activation='softmax')
    def call(self,x, training=None, mask=None):
        x = self.d1(x)
        return self.d2(x)

### 학습 루프 정의

In [26]:
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_metric):
    with tf.GradientTape() as tape:
        predictions = model(inputs)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables) # grad(loss), df(x)/dx
    
    optimizer.apply_gradients(zip(gradients,model.trainable_variables))
    train_loss(loss)
    train_metric(labels, predictions)

### 데이터셋 생성, 전처리

In [27]:
np.random.seed(0)

pts = list()
labels = list()
center_pts = np.random.uniform(-8.0,8.0,(10,2))
for label, center_pt in enumerate(center_pts):
    for _ in range(100):
        pts.append(center_pt + np.random.randn(*center_pt.shape))
        labels.append(label)
        
pts = np.stack(pts,axis=0).astype(np.float32)
labels = np.stack(labels,axis=0)

train_ds = tf.data.Dataset.from_tensor_slices((pts,labels)).shuffle(1000).batch(32)

### 모델 생성

In [28]:
model = MyModel()

## 손실 함수 및 최적화 알고리즘 설정

In [29]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

## 평가 지표 설정

### Accuracy

In [30]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

## 학습 루프

In [31]:
for epoch in range(EPOCHS):
    for x, label in train_ds:
        train_step(model, x, label, loss_object, optimizer, train_loss, train_accuracy)
        
    template = 'Epoch {}, Loss : {}, Accuracy : {}'
    print(template.format(epoch+1,train_loss.result(),train_accuracy.result()*100))

Epoch 1, Loss : 2.28741717338562, Accuracy : 19.5
Epoch 2, Loss : 2.0835788249969482, Accuracy : 33.349998474121094
Epoch 3, Loss : 1.9359787702560425, Accuracy : 41.83333206176758
Epoch 4, Loss : 1.8153080940246582, Accuracy : 47.67499923706055
Epoch 5, Loss : 1.7158801555633545, Accuracy : 52.119998931884766
Epoch 6, Loss : 1.6340006589889526, Accuracy : 55.5
Epoch 7, Loss : 1.5626691579818726, Accuracy : 58.61428451538086
Epoch 8, Loss : 1.5000571012496948, Accuracy : 60.66250228881836
Epoch 9, Loss : 1.4435627460479736, Accuracy : 63.0
Epoch 10, Loss : 1.392846941947937, Accuracy : 64.76000213623047
Epoch 11, Loss : 1.347184419631958, Accuracy : 66.49090576171875
Epoch 12, Loss : 1.3045928478240967, Accuracy : 67.98333740234375
Epoch 13, Loss : 1.2664687633514404, Accuracy : 69.26922607421875
Epoch 14, Loss : 1.2310278415679932, Accuracy : 70.37857055664062
Epoch 15, Loss : 1.1979495286941528, Accuracy : 71.5133285522461
Epoch 16, Loss : 1.1671541929244995, Accuracy : 72.4312515258

## 데이터셋 및 학습 파라미터 저장

In [ ]:
np.savez_compressed('ch2_dataset.npz',inputs=pts,labels=labels)

W_h, b_h = model.d1.get_weights()
W_o, b_o = model.d2.get_weights()
W_h = np.transpose(W_h)
W_o = np.transpose(W_o)
np.savez_compressed('ch2_parameters.npz',
                   W_h = W_h,
                   b_h = b_h,
                   W_o = W_o,
                   b_o = b_o)